### INSPECTION

In [1]:
import pandas as pd
import os

In [ ]:
# 1. Setup File Paths (Using raw strings r'' for Windows paths)
files = {
    "customers": r"D:\Data_Science\Projects\Portfolio_Projects\Olist-Logistics-Analysis\data\raw\olist_customers_dataset.csv",
    "geolocation": r"D:\Data_Science\Projects\Portfolio_Projects\Olist-Logistics-Analysis\data\raw\olist_geolocation_dataset.csv",
    "orders": r"D:\Data_Science\Projects\Portfolio_Projects\Olist-Logistics-Analysis\data\raw\olist_orders_dataset.csv",
    "order_items": r"D:\Data_Science\Projects\Portfolio_Projects\Olist-Logistics-Analysis\data\raw\olist_order_items_dataset.csv",
    "order_payments": r"D:\Data_Science\Projects\Portfolio_Projects\Olist-Logistics-Analysis\data\raw\olist_order_payments_dataset.csv",
    "order_reviews": r"D:\Data_Science\Projects\Portfolio_Projects\Olist-Logistics-Analysis\data\raw\olist_order_reviews_dataset.csv",
    "products": r"D:\Data_Science\Projects\Portfolio_Projects\Olist-Logistics-Analysis\data\raw\olist_products_dataset.csv",
    "sellers": r"D:\Data_Science\Projects\Portfolio_Projects\Olist-Logistics-Analysis\data\raw\olist_sellers_dataset.csv",
    "category_translation": r"D:\Data_Science\Projects\Portfolio_Projects\Olist-Logistics-Analysis\data\raw\product_category_name_translation.csv"
}

def inspect_dataset(name, file_path):
    print("="*60)
    print(f"📊 DATASET: {name.upper()}")
    print("="*60)
    
    if not os.path.exists(file_path):
        print(f"❌ ERROR: File not found at {file_path}")
        return

    try:
        df = pd.read_csv(file_path)
        
        # 1. Shape & Duplicates
        print(f"• Rows: {df.shape[0]} | Columns: {df.shape[1]}")
        print(f"• Duplicates: {df.duplicated().sum()}")
        
        # 2. Missing Values (Only show columns that actually have missing data)
        missing = df.isnull().sum()
        missing = missing[missing > 0]
        if not missing.empty:
            print("\n⚠️  MISSING VALUES:")
            print(missing)
        else:
            print("\n✅ No missing values found.")

        # 3. Data Types (Crucial for SQL/Power BI prep)
        print("\nTypes:")
        print(df.dtypes)
        
        # 4. Preview
        print("\n👀 Head (First 3 rows):")
        print(df.head(3))
        print("\n")
        
    except Exception as e:
        print(f"❌ ERROR reading file: {e}")

# Run the inspection loop
for name, path in files.items():
    inspect_dataset(name, path)

📊 DATASET: CUSTOMERS
• Rows: 99441 | Columns: 5
• Duplicates: 0

✅ No missing values found.

Types:
customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

👀 Head (First 3 rows):
                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   

   customer_zip_code_prefix          customer_city customer_state  
0                     14409                 franca             SP  
1                      9790  sao bernardo do campo             SP  
2                      1151              sao paulo             SP  


📊 DATASET: GEOLOCATION
• Rows: 1000163 | Columns: 5
• Duplicates: 261831

✅ No missing values found.

Types:
geolocation

### CLEANING & TRANSFORMATION

In [ ]:
# 1. Load Dataframes into memory (since the previous step only printed them)
dfs = {}
for name, path in files.items():
    dfs[name] = pd.read_csv(path)
    print(f"Loaded: {name}")

print("-" * 30)

# ---------------------------------------------------------
# TRANSFORM 1: Fix Date Types
# ---------------------------------------------------------
print("Processing Dates...")
date_cols = [
    'order_purchase_timestamp', 
    'order_approved_at', 
    'order_delivered_carrier_date', 
    'order_delivered_customer_date', 
    'order_estimated_delivery_date'
]

# Convert to datetime (errors='coerce' turns bad data into NaT/Null instead of crashing)
for col in date_cols:
    dfs['orders'][col] = pd.to_datetime(dfs['orders'][col], errors='coerce')

# Do the same for order_items and reviews
dfs['order_items']['shipping_limit_date'] = pd.to_datetime(dfs['order_items']['shipping_limit_date'], errors='coerce')
dfs['order_reviews']['review_creation_date'] = pd.to_datetime(dfs['order_reviews']['review_creation_date'], errors='coerce')
dfs['order_reviews']['review_answer_timestamp'] = pd.to_datetime(dfs['order_reviews']['review_answer_timestamp'], errors='coerce')

print("✅ Dates converted successfully.")

# ---------------------------------------------------------
# TRANSFORM 2: Fix Geolocation Duplicates
# ---------------------------------------------------------
print("Processing Geolocation...")
# We group by Zip Code and take the first Lat/Lng found. 
# This reduces rows from 1M+ to just unique Zip Codes (~19k).
geo_clean = dfs['geolocation'].groupby('geolocation_zip_code_prefix').first().reset_index()
dfs['geolocation'] = geo_clean
print(f"✅ Geolocation duplicates removed. New Shape: {dfs['geolocation'].shape}")

# ---------------------------------------------------------
# TRANSFORM 3: Merge Product Translations & Fix Missing Categories
# ---------------------------------------------------------
print("Processing Products...")

# Fill missing category names in Portuguese with 'outros' (others) to avoid join errors
dfs['products']['product_category_name'] = dfs['products']['product_category_name'].fillna('outros')

# Merge with the translation table (Left Join ensures we don't lose products if translation is missing)
products_merged = pd.merge(
    dfs['products'], 
    dfs['category_translation'], 
    on='product_category_name', 
    how='left'
)

# If translation is missing (NaN), fill English column with the Portuguese name (better than nothing)
products_merged['product_category_name_english'] = products_merged['product_category_name_english'].fillna(products_merged['product_category_name'])

# Update the dataframe in our dictionary
dfs['products'] = products_merged

# Drop the portuguese column now? Optional, but keeps it clean.
# Let's keep it for now just in case, but rely on English for the DB.
print("✅ Products translated to English.")

# ---------------------------------------------------------
# FINAL CHECK
# ---------------------------------------------------------
print("=" * 30)
print(f"Orders Date Type Check: {dfs['orders']['order_purchase_timestamp'].dtype}")
print(f"Geolocation Unique Zips: {dfs['geolocation'].shape[0]}")
print(f"Products with English Category: {dfs['products']['product_category_name_english'].notnull().sum()} / {dfs['products'].shape[0]}")

Loaded: customers
Loaded: geolocation
Loaded: orders
Loaded: order_items
Loaded: order_payments
Loaded: order_reviews
Loaded: products
Loaded: sellers
Loaded: category_translation
------------------------------
Processing Dates...
✅ Dates converted successfully.
Processing Geolocation...
✅ Geolocation duplicates removed. New Shape: (19015, 5)
Processing Products...
✅ Products translated to English.
Orders Date Type Check: datetime64[ns]
Geolocation Unique Zips: 19015
Products with English Category: 32951 / 32951


### DEEP LOGICAL INSPECTION

In [ ]:
# ---------------------------------------------------------
# CHECK 1: TIME TRAVEL (Business Logic)
# ---------------------------------------------------------
# Do we have orders delivered BEFORE they were purchased? (Impossible)
invalid_dates = dfs['orders'][dfs['orders']['order_delivered_customer_date'] < dfs['orders']['order_purchase_timestamp']]
print(f"1️⃣ Time Travel Errors (Delivered < Purchased): {len(invalid_dates)} rows")

if len(invalid_dates) > 0:
    print("   -> displaying first 3 examples:")
    print(invalid_dates[['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date']].head(3))

# ---------------------------------------------------------
# CHECK 2: PRICE ANOMALIES
# ---------------------------------------------------------
# Do we have items given away for free or negative prices?
free_items = dfs['order_items'][dfs['order_items']['price'] <= 0]
print(f"\n2️⃣ Price Anomalies (Price <= 0): {len(free_items)} rows")

# ---------------------------------------------------------
# CHECK 3: REFERENTIAL INTEGRITY (Crucial for SQL Foreign Keys)
# ---------------------------------------------------------
# Check: Do all Order Items belong to an Order that actually exists?
# If this fails, SQL will reject the data when we try to create Foreign Keys.
items_orders = set(dfs['order_items']['order_id'])
actual_orders = set(dfs['orders']['order_id'])
orphans = items_orders - actual_orders # Items referencing an order_id that doesn't exist in the orders table

print(f"\n3️⃣ Orphaned Order Items: {len(orphans)}")
if len(orphans) > 0:
    print("   ⚠️ CRITICAL: We have items for orders that don't exist in the Orders table.")

# Check: Do all Orders belong to a Customer that actually exists?
order_custs = set(dfs['orders']['customer_id'])
actual_custs = set(dfs['customers']['customer_id'])
orphan_custs = order_custs - actual_custs

print(f"4️⃣ Orphaned Orders (No Customer Found): {len(orphan_custs)}")

# ---------------------------------------------------------
# CHECK 4: PRODUCT CATEGORY COMPLETENESS
# ---------------------------------------------------------
# Did our previous merge actually work for everything?
missing_trans = dfs['products']['product_category_name_english'].isnull().sum()
print(f"\n5️⃣ Products without English Category: {missing_trans}")

1️⃣ Time Travel Errors (Delivered < Purchased): 0 rows

2️⃣ Price Anomalies (Price <= 0): 0 rows

3️⃣ Orphaned Order Items: 0
4️⃣ Orphaned Orders (No Customer Found): 0

5️⃣ Products without English Category: 0


### EXPORT TO LOCAL DRIVE

In [ ]:
# 1. Define the export folder path
export_path = r"D:\Data_Science\Projects\Portfolio_Projects\Olist-Logistics-Analysis\data\processed"

# 2. Create the folder if it doesn't exist (Safety check)
os.makedirs(export_path, exist_ok=True)

print("\n" + "="*30)
print(f"📂 EXPORTING CLEAN DATA TO:\n{export_path}")
print("="*30)

# 3. Iterate through the dictionary and save each dataframe
for name, df in dfs.items():
    # Create a filename, e.g., 'orders.csv'
    # We use the original key name 'orders', 'products', etc.
    filename = f"{name}.csv"
    full_path = os.path.join(export_path, filename)
    
    # Export to CSV
    # index=False prevents pandas from adding that annoying 0,1,2,3... column
    df.to_csv(full_path, index=False)
    
    print(f"✅ Saved: {filename} ({df.shape[0]} rows)")


📂 EXPORTING CLEAN DATA TO:
D:\Data_Science\Projects\Portfolio_Projects\Olist-Logistics-Analysis\data\processed
✅ Saved: customers.csv (99441 rows)
✅ Saved: geolocation.csv (19015 rows)
✅ Saved: orders.csv (99441 rows)
✅ Saved: order_items.csv (112650 rows)
✅ Saved: order_payments.csv (103886 rows)
✅ Saved: order_reviews.csv (99224 rows)
✅ Saved: products.csv (32951 rows)
✅ Saved: sellers.csv (3095 rows)
✅ Saved: category_translation.csv (71 rows)
